In [7]:
import os, sys
from pathlib import Path
script_dir = Path(os.path.dirname(os.path.abspath('')))
module_dir = str(script_dir)
sys.path.insert(0, module_dir + '/modules')
print(module_dir)

# import the rest of the modules
%matplotlib nbagg
import numpy as np
import tensorflow as tf 
import matplotlib.pyplot as plt
import arch

C:\Users\pinak\Documents\GitHub\non-grad3D


In [13]:
r = 1.
X = np.random.normal(size=(5, 3)).astype('float32')

In [3]:
X

array([[ 0.49909775,  0.59041585, -1.59777827],
       [ 0.82492991,  0.66685465,  2.32737369],
       [-1.94871424,  0.0508806 ,  0.50802789],
       [ 1.43583623, -1.80601372, -0.29170615],
       [-0.65784376, -1.01578823, -1.03879669]])

In [4]:
x, y, z = tf.split(X, [1, 1, 1], axis=-1)

In [5]:
x

<tf.Tensor: shape=(5, 1), dtype=float64, numpy=
array([[ 0.49909775],
       [ 0.82492991],
       [-1.94871424],
       [ 1.43583623],
       [-0.65784376]])>

In [8]:
def get_net(iter):
    net = arch.LSTMForgetNet(num_nodes=50, num_blocks=3)
    if iter < 500:
        net.load_weights('../data/Thomas-true-vs-learned/init/Thomas_{}'.format(iter)).expect_partial()
    elif iter < 10000:
        net.load_weights('../data/Thomas-true-vs-learned/Thomas_{}'.format(iter)).expect_partial()
    else:
        net.load_weights('../data/Thomas/400k/Thomas').expect_partial()
    return net


n_theta = get_net(1000000)

def h0(X):
    l = len(X)
    m = int(1e5)
    M = int(np.ceil(l / m))
    data = []
    for i in range(M):
        if i < M-1:
            x_, y_, z_ = tf.split(X[i*m: (i+1)*m], [1, 1, 1], axis=-1)
        else:
            x_, y_, z_ = tf.split(X[i*m:], [1, 1, 1], axis=-1)
        log_p0 = (- (x_**2 + y_**2 + z_**2) / (2.*r**2)).numpy()
        log_pinf = n_theta(x_, y_, z_).numpy()
        data.append(np.exp(log_p0 - log_pinf) / (2. * np.pi * r**2) ** (1.5))
        
    return np.concatenate(data, axis=0)

In [14]:
h0(X)

array([[0.01570974],
       [0.01246196],
       [0.01768144],
       [0.00354082],
       [0.00762022]], dtype=float32)